# LLM Examples
This notebook will showcase some examples of LLMs and how to use them. 

Some of the boilerplate code is hidden in the "src" directory - go have a look!

In [1]:
!git clone https://github.com/SamHollings/llm_examples.git

Cloning into 'llm_examples'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 12), reused 45 (delta 4), pack-reused 0
Unpacking objects: 100% (59/59), 61.38 KiB | 3.07 MiB/s, done.


In [1]:
!pip install -r llm_examples/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117


In [2]:
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

## Test GPU enabled

In [2]:
torch.cuda.is_available()

True

If the above shows false, trying a cuda tensor will show a more informative error message (such as the non-CUDA enabled version of PyTorch being installed)

In [3]:
a=torch.cuda.FloatTensor()

## Dolly

Followed instructions on:

- https://github.com/databrickslabs/dolly
- https://huggingface.co/databricks/dolly-v2-12b#dolly-v2-12b-model-card

In [6]:
# get and save the model
# load the model
# use the model on a string
# use the model on a dataframe
# use the model on a spark dataframe

In [3]:
model_name = "databricks/dolly-v2-3b"

The below will try and load the model from hugging face, but if a local directory exists with the same name, it will try and use that.

In [4]:
instruct_pipeline = pipeline(
model=model_name, #3b, 7b, 12b 
torch_dtype=torch.float16, #bfloat16 
trust_remote_code=True, 
device_map="auto",
#model_kwargs={'load_in_8bit': True},
)

The below will load the model from a local drive if it is present - simply download the model files from https://huggingface.co/databricks/dolly-v2-3b/tree/main and then put them in a directory called "models/dtaabricks/dolly-v2-3b)

In [8]:
# instruct_pipeline = pipeline(
# model=f"models/{model_name}", #3b, 7b, 12b 
# torch_dtype=torch.float16, #bfloat16 
# trust_remote_code=True, 
# device_map="auto",
# )

Here we apply the model to some simply challenges - generating words and generating haikus - note this isn't the most effcient way of using the GPU as this is being done sequentially - so we need to look into using datasets, or spark drames and distributed inference.

In [51]:
import pandas as pd
import numpy as np
import random
import string

df = pd.DataFrame(np.random.choice(list(string.ascii_letters),10,1), columns=['Input',])

generate_word = lambda x: instruct_pipeline(f"Generate a word starting with the letter {x}. Return only this word.")[0]['generated_text']

generate_haiku = lambda x: instruct_pipeline(f"Generate a haiku starting with the letter {x}. Return only this haiku.")[0]['generated_text']


df['Word'] = df["Input"].apply(generate_word)
df['Haiku'] = df["Input"].apply(generate_haiku)
df

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,Input,Word,Haiku
0,l,letter,Lovelier than a rose by a stream\nPure as a pr...
1,T,term,Tenderly you hold my hand\nAs we stroll throug...
2,j,Judgement,Whispering idols\n\nFeather boa and pearls of ...
3,X,taxonomy,Panda for ever\nIs best when in need\nOf an ex...
4,f,foo,fivviteshfatiroke\nmydreamscometrue
5,w,Word starting with letter w - return\nWhisky\n...,"Whistle and tinkle, the kittens play with the ..."
6,T,terminate,Tundra\nFur seal and polar bear\nChat on about...
7,m,mammal,I hold the mirror up to nature\nTo see her ess...
8,C,cyclotomic,"Ca, ca, caughee\nCa (for ""cold""), ca (for ""sno..."
9,Z,zombie,"I stare at the stars, \nall knowing Z is out ..."


In [10]:
instruct_pipeline("Explain to me the difference between nuclear fission and fusion.")

[{'generated_text': 'Fission creates one atom and a fragment of an atom. Fusion creates many atoms and a lot of energy as well.'}]

In [52]:
print(instruct_pipeline("Explain the history of the united kingdom")[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


The history of the United Kingdom can be divided into three phases: 
First Phase - 1558-1753 - During this time the English Crown governed England alone.
Second Phase - 1753-1801 - The British Empire was unified through the Act of Union 1707 which unified the realms of England, Scotland and Wales into the Kingdom of Great Britain. 
Third Phase - 1801-present - the Act of Union with Ireland abolished.
